In [1]:
# Run this again after editing submodules so Colab uses the updated versions
from citylearn import  CityLearn
from pathlib import Path
from agent import Agent
import numpy as np                                                                                                                                                                                      
import torch

In [25]:
# Load environment
climate_zone = 5
params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)), 
        'building_attributes':'building_attributes.json', 
        'weather_file':'weather_data.csv', 
        'solar_profile':'solar_generation_1kW.csv', 
        'carbon_intensity':'carbon_intensity.csv',
        'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
        'buildings_states_actions':'buildings_state_action_space.json', 
        'simulation_period': (0, 8760*4-1), 
        'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'], 
        'central_agent': False,
        'save_memory': False }

# Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
env = CityLearn(**params)
observations_spaces, actions_spaces = env.get_state_action_spaces()

# Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
building_info = env.get_building_information()

In [30]:
len(actions_spaces)

9

In [13]:
params_agent = {'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
                 'buildings_states_actions':'buildings_state_action_space.json', 
                 'building_info':building_info,
                 'observation_spaces':observations_spaces, 
                 'action_spaces':actions_spaces}

# Instantiating the control agent(s)
agents = Agent(**params_agent)

state = env.reset()
done = False

action, coordination_vars = agents.select_action(state)    
while not done:
    next_state, reward, done, _ = env.step(action)
    action_next, coordination_vars_next = agents.select_action(next_state)
    agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)
    coordination_vars = coordination_vars_next
    state = next_state
    action = action_next

env.cost()

Device:cuda


({'ramping': 1.6146312,
  '1-load_factor': 1.1158112003254197,
  'average_daily_peak': 1.1334853,
  'peak_demand': 1.1599934,
  'net_electricity_consumption': 1.0157715,
  'carbon_emissions': 1.0229925,
  'total': 1.177114183933326,
  'coordination_score': 1.2559802759152476},
 {'ramping_last_yr': 1.1639097,
  '1-load_factor_last_yr': 1.0558413421324109,
  'average_daily_peak_last_yr': 1.0668069,
  'peak_demand_last_yr': 1.1332848,
  'net_electricity_consumption_last_yr': 1.0071987,
  'carbon_emissions_last_yr': 1.0151038,
  'coordination_score_last_yr': 1.1049606838626467,
  'total_last_yr': 1.0781579896667743})

In [7]:
def make(x,y):
    print(f"making {x} envs; \n"
          f" and {y} threads")
make(42,3)

making 42 envs; 
 and {y} threads


In [14]:
for obs_space, act_space in zip(observations_spaces, actions_spaces):
    print(act_space.shape[0])

3
3
2
2
3
3
3
3
3


In [19]:
obs = env.reset()
observations_spaces = env.observation_spaces
actions_spaces = env.action_spaces

In [20]:
print(observations_spaces)

[Box([ 1.    1.    1.   -4.15 -4.41 -4.72 -5.34 14.38 14.33 14.29 14.19  0.
 -0.    0.    0.    0.   -0.    0.    0.   16.8  11.17  7.42  0.    0.
  0.    0.    0.    0.  ], [1.2000000e+01 8.0000000e+00 2.4000000e+01 3.6700001e+01 3.6970001e+01
 3.7279999e+01 3.7910000e+01 1.0000000e+02 1.0250000e+02 1.0500000e+02
 1.0999000e+02 4.9162000e+02 5.0142001e+02 5.1122000e+02 5.3081000e+02
 1.2126200e+03 1.2308300e+03 1.2532200e+03 1.2979900e+03 2.6379999e+01
 9.2389999e+01 7.0910004e+01 9.9999962e+01 1.0000000e+00 1.0000000e+00
 1.0000000e+00 3.3668436e+02 6.8709970e-01], (28,), float32), Box([ 1.    1.    1.   -4.15 -4.41 -4.72 -5.34 14.38 14.33 14.29 14.19  0.
 -0.    0.    0.    0.   -0.    0.    0.   18.91  7.01  1.92  0.    0.
  0.    0.    0.    0.  ], [1.200000e+01 8.000000e+00 2.400000e+01 3.670000e+01 3.697000e+01
 3.728000e+01 3.791000e+01 1.000000e+02 1.025000e+02 1.050000e+02
 1.099900e+02 4.916200e+02 5.014200e+02 5.112200e+02 5.308100e+02
 1.212620e+03 1.230830e+03 1.253220e+0

In [24]:
obs = env.reset()
ob = obs[2]
print(len(ob))

27


In [22]:
[obsp.shape[0] for obsp in observations_spaces]

[28, 28, 27, 28, 28, 28, 28, 28, 28]